In [ ]:
import os
import sys
from pathlib import Path
from typing import Tuple, List
import itertools

import numpy as np
import pandas as pd
import cv2
import pytesseract
from PIL import Image

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Add package to system path.
sys.path.insert(0, "../mystique/")

In [ ]:
# Image resize keepint the aspect ratio of the image.
# 1. Upscale size of 800
size = np.array((450, 200)) # w, h
fact = (800 / 450)
print (size[0] / size[1], size)
new_size = size * fact
print(new_size[0]/new_size[1], new_size)

# 2. Down scale to max size 1300

In [ ]:
template_labels1 = pd.read_csv("/home/haridas/projects/mystique/data/train_and_test-2020-Jun-05-coco/templates_test_data_coco.csv")

In [ ]:
template_labels = pd.read_csv("/home/haridas/projects/mystique/data/templates_test_data.csv")
template_imgdir = Path("/home/haridas/projects/mystique/data/templates_test_data/")

In [ ]:
# template_labels.groupby("filename").count()

In [ ]:
template_labels = template_labels[template_labels["class"].isin(["textbox", "actionset", "radiobutton"])]

In [ ]:
def crop_prop(row: pd.Series, img_dir: str):
    img = Image.open(Path(img_dir) / Path(row.filename))
    #box_labels = labels[labels.filename == "2.png"]
    # img.crop(box_labels.tolist()[-4:]).save("cropped.png")
    #for row in box_labels.iterrows():
    imw, imh = img.size
    box_coord = row.tolist()[-4:]
    img = img.crop(box_coord)
    w, h = img.size
    names = ("filename", "imwidth", "imheight", 
             "bwidth", "bheight", "left", "top", "right", "bottom",
             #"cwidth", "cheight", "cw_iw", "ch_ih",
             "char", "cleft", "cbottom", "cright", "ctop",
             "box_srno")
    # remove page and add index to box.
    char_bboxs = [dict(list(zip(names,
                                [row.filename, imw, imh, w, h] + 
                                box_coord + 
                                box.split()[:-1] + 
                                [row.name]
                               )))
                  for ind,box in enumerate(pytesseract.image_to_boxes(img).split("\n")) if box.split()]
    
    # bottom-left,right-top => top-left, bottom-right
    for cbox in char_bboxs:
        cbox["ctop"] = h - int(cbox["ctop"])
        cbox["cbottom"] = h - int(cbox["cbottom"])
    return char_bboxs

In [ ]:
# pd.DataFrame.from_records?
# template_labels.apply
img = Image.open(template_imgdir / "Group 5.png").convert("RGB")

In [ ]:
# plt.plot(np.asarray(img))
plt.imshow(img)

In [ ]:
# img.crop?
# template_labels.head()
img_np = np.array(img)

In [ ]:
# cv2.cvtColor?

In [ ]:
# RGB -> BGR 
img_np[:, :, ::-1]
# cv2.cvtColor(img_np, cv2.CV2)

In [ ]:
# img_t = Image.open("../mystique/models/pth/detr_cpp/cropped_images/cropped.png")
chardf = pd.DataFrame.from_records(list(itertools.chain.from_iterable(
    template_labels.apply(lambda x: crop_prop(x, template_imgdir), axis=1))))

In [ ]:
chardf.head()

In [ ]:
chardf.cleft = chardf.cleft.apply(lambda x: int(x))
chardf.ctop = chardf.ctop.apply(lambda x: int(x))
chardf.cright = chardf.cright.apply(lambda x: int(x))
chardf.cbottom = chardf.cbottom.apply(lambda x: int(x))

In [ ]:
#"cwidth", "cheight", "cw_iw", "ch_ih",
chardf["cwidth"] = chardf["cright"] - chardf["cleft"]
chardf["cheight"] = chardf["cbottom"] - chardf["ctop"]
chardf["cw_iw"] = (chardf["cwidth"] / chardf["imwidth"]) * 100 
chardf["ch_ih"] = (chardf["cheight"] / chardf["imheight"]) * 100

In [ ]:
names = ["filename", "imwidth", "imheight", 
         "bwidth", "bheight", "left", "top", "right", "bottom",
         "cwidth", "cheight", "cw_iw", "ch_ih",
         "char", "cleft", "ctop", "cright", "cbottom",
         "box_srno"]
chardf = chardf[names]

In [ ]:
chardf.shape

In [ ]:
chardf.head()

In [ ]:
chardf.to_csv("./template_chars_map_new.csv", index=False)

In [ ]:
def gbf_save_cropped_img(imgdf: pd.DataFrame, img_dir, out_file="cropped.png"):
    "For every box, we get the list of char and its properties."
    row = imgdf.iloc[0]
    imgname = row.filename
    box_srno = row.box_srno
    left, top, right, bottom = row.left, row.top, row.right, row.bottom
    img = Image.open(Path(img_dir) / Path(imgname)).convert("RGB")
    img = img.crop((left, top, right, bottom))
    
    img_np = np.array(img)
    
    # RGB -> BGR, cv2 comptible, copy required keep the layout correct.
    img_cv2 = img_np[:, :, ::-1].copy()
    
    for ind, crow in imgdf.iterrows():
        _ = cv2.rectangle(
            img_cv2,
            (int(crow.cleft), int(crow.ctop)),
            (int(crow.cright), int(crow.cbottom)),
            (0,0,0),
            1)
    cv2.imwrite(f"{imgname}_{box_srno}_{out_file}", img_cv2)
    
    
def gbf_box_stats(imgdf: pd.DataFrame):
    "For every box, we get the list of char and its properties."
    row = imgdf.iloc[0]
    imgname = row.filename
    box_srno = row.box_srno
    
    test = imgdf.describe()
    print(test)

In [ ]:
# Plot images for few samples.
# chardf.groupby(by=["filename", "box_srno"]).apply(lambda x: gbf_save_cropped_img(x, img_dir=template_imgdir))

In [ ]:
chardf[((chardf.box_srno == 0) & (chardf.filename == "Group 5.png"))].describe(np.arange(0, 1, 0.1))[["cw_iw", "ch_ih"]][1:].plot(kind="bar", table=True)

In [ ]:
chardf.describe(np.arange(0, 1, 0.1))[["cw_iw", "ch_ih"]][1:]

In [ ]:
chardf.iloc[:100].groupby(by=["filename", "box_srno"]).apply(gbf_box_stats)